# An animation demo

Matplolib gives us two ways to make animations:
- [FuncAnimation](https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation) which makes an animation by repeatedly calling a function func, and
- [ArtistAnimation](https://matplotlib.org/api/_as_gen/matplotlib.animation.ArtistAnimation.html#matplotlib.animation.ArtistAnimation) which is an animation using a fixed set of Artist objects.

In this notebook, we look at both `FuncAnimation` and `ArtistAnimation` as examples.

**Nota Bene** these examples need `%matplolib notebook` or `%matplotlib nbagg` enabled _instead of_ `%matplotlib inline` in order to render in the notebook.

In [ ]:
# set the matplolib backend to use
%matplotlib notebook
#%matplotlib nbagg
# imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation
import math

## Example from the Documentation

Here is a [link to the documentation](https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation) for quick reference.

### Rain simulation

Modified from source: [Rain simulation](https://matplotlib.org/gallery/animation/rain.html#sphx-glr-gallery-animation-rain-py)

Author: Nicolas P. Rougier

In [ ]:
# Fixing random state for reproducibility
np.random.seed(42)

# Create new Figure and an Axes which fills it.
fig = plt.figure(figsize=(3, 3))
ax = fig.add_axes([0, 0, 1, 1], frameon=False)

# Create rain data
n_drops = 50
rain_drops = np.zeros(n_drops, dtype=[('position', float, 2),
                                      ('size',     float, 1),
                                      ('growth',   float, 1),
                                      ('color',    float, 4)])

# Initialize the raindrops in random positions and with
# random growth rates.
rain_drops['position'] = np.random.uniform(0, 1, (n_drops, 2))
rain_drops['growth'] = np.random.uniform(50, 200, n_drops)

# Construct the scatter which we will update during animation
# as the raindrops develop.
scat = ax.scatter(rain_drops['position'][:, 0], rain_drops['position'][:, 1],
                  s=rain_drops['size'], lw=0.5, edgecolors=rain_drops['color'],
                  facecolors='none')

def update(frame_number):
    # Get an index which we can use to re-spawn the oldest raindrop.
    current_index = frame_number % n_drops

    # Make all colors more transparent as time progresses.
    rain_drops['color'][:, 3] -= 1.0/len(rain_drops)
    rain_drops['color'][:, 3] = np.clip(rain_drops['color'][:, 3], 0, 1)

    # Make all circles bigger.
    rain_drops['size'] += rain_drops['growth']

    # Pick a new position for oldest rain drop, resetting its size,
    # color and growth factor.
    rain_drops['position'][current_index] = np.random.uniform(0, 1, 2)
    rain_drops['size'][current_index] = 5
    rain_drops['color'][current_index] = (0, 0, 0, 1)
    rain_drops['growth'][current_index] = np.random.uniform(50, 200)

    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_edgecolors(rain_drops['color'])
    scat.set_sizes(rain_drops['size'])
    scat.set_offsets(rain_drops['position'])


# Construct the animation, using the update function as the animation director.
animation = FuncAnimation(fig, update, interval=10)
plt.show()

## Deconstructing the code

What are the moving parts in the example above?

First it should be noted from the [docs](https://matplotlib.org/api/animation_api.html) that:
>[...] it is critical to keep a reference to the instance object. The animation is advanced by a timer (typically from the host GUI framework) which the Animation object holds the only reference to. If you do not hold a reference to the Animation object, it (and hence the timers), will be garbage collected which will stop the animation.

This is achieved in the _rain simulation_ example by saving the `fig` object which is passed to the `FuncAnimation()` function.

In this example, the `FuncAnimation()` function takes three arguments:

`animation = FuncAnimation(fig, update, interval=10)`

1. `fig` which is the matplotlib figure object
2. an `update` function that will be repeatedly called
3. a keyword argument `interval` specifiying the delay between frames in milliseconds

## An example from scratch

We first define a function to get points on the perimeter of a circle:

In [ ]:
def perimeter(radius=1,n_points=36):
    """for a given radius r, calculate the coordinates of n points on the perimeter of a circle"""
    points = []
    for point in range(0,n_points):
        x = math.cos(2 * math.pi / n_points * point) * radius
        y = math.sin(2 * math.pi / n_points * point) * radius
        points.append((x,y))
    
    return np.array(points)

We then build a matplotlib plot (this must be done for the animation to work; the plot must be created once):

In [ ]:
# initialize the data
radius, n_points = 1, 12
x = perimeter(radius,n_points)[:,0]
y = perimeter(radius,n_points)[:,1]

# create the figure
fig, ax = plt.subplots(figsize=(4,4), ncols=1)

# set up the colors
alphas = np.linspace(0.1, 1, n_points)
rgba_colors = np.zeros((n_points,4))
rgba_colors[:, 3] = alphas

# create and display the plot
ax.scatter(x,y, color=rgba_colors)
ax.set_xticks(np.array(range(-10,11,5)) / 10)
ax.set_yticks(np.array(range(-10,11,5)) / 10)
ax.grid()
ax.set_title(f'Perimeter plot')

plt.show()

Now we have this, we can save this plot initialization into a function:

In [ ]:
def initialize_plot(radius, n_points):
    """
    initialize a matplotlib plot of given radius with
    n_points regularly spaced along its perimeter
    """
    # Set plot ticks
    ticks = np.array(range(-10,11,5)) / 10

    # Create new Figure and Axis
    fig, ax = plt.subplots(figsize=(4,4), ncols=1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_title(f'Perimeter plot')
    ax.grid()

    # Create x, y points
    x = perimeter(radius, n_points)[:,0]
    y = perimeter(radius, n_points)[:,1]

    # Initialize alphas and rgba colors
    alphas = np.linspace(0.1, 1, n_points)
    rgba_colors = np.zeros((n_points,4))
    rgba_colors[:, 3] = alphas

    # Construct the scatter which we will update during animation
    scat = ax.scatter(x, y, c=rgba_colors)
    
    return fig, scat

It's now time to build the function that will be called repeatedly by `FuncAnimation`, we use `np.roll()` to [_roll_](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.roll.html) the alphas array at each time step and use this to set the point sizes too:

In [ ]:
def rotate(frame_number, scat, alphas):
    """
    rotates the alpha linspace, resets the rgba_colors,
    sets facecolors and sizes to scat
    
    returns updated scat
    """
    alphas = np.roll(alphas,frame_number)
    rgba_colors[:, 2] = alphas
    size = alphas
    # Update the scatter collection, with the new colors
    scat.set_facecolors(rgba_colors)
    scat.set_sizes(alphas * 300)
    
    return scat

We can now call `initialize_plot()` and save it to `(fig, scat)` and then save `FuncAnimation()` to a `name` and pass it `fig`, the `rotate()` function and the requirted `fargs` (see [docs](https://matplotlib.org/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation))

In [ ]:
# Initialize the figure
fig, scat = initialize_plot(radius=1, n_points=12)

# Construct the animation, using the rotate function as the animation director.
anim = FuncAnimation(fig, rotate, fargs=[scat, alphas], interval=150, blit=True)
plt.show()

We can now save this animated plot to a \*.gif for example:

In [ ]:
anim.save('data/rolling_points.gif', writer='imagemagick', fps=30)

## Subsurface examples

### An animated Ricker wavelet

In [ ]:
from bruges.filters import ricker

In [ ]:
# intialize the plot
fig, ax = plt.subplots(figsize=(8, 4), ncols=1)

# initialize the data
duration, dt, frequency = 0.128, 0.001, 25
freqs_up = np.arange(20,50,1)
freqs_down = np.array(sorted(freqs_up, reverse=True))[1:-1]
freqs = np.append(freqs_up, freqs_down)

# create the ricker wavelet
w, t = ricker(duration=duration, dt=dt, f=frequency, return_t=True)

# instantiate the line plot
line, = ax.plot(t, w)

# define the init() function that is passed the FuncAnimation()
def init():
    line.set_xdata(t)
    line.set_ydata(w)
    ax.grid()

    return line,

# define the animate() function that is passed to FuncAnimation()
def animate(frame):
    frequencies = np.roll(freqs, frame)
    frequency = frequencies[0]
    
    w, t = ricker(duration=duration, dt=dt, f=frequency, return_t=True)

    line.set_xdata(t)
    line.set_ydata(w)
    
    ax.set_title(f'Ricker wavelet - frequency={frequency}')
    
    return line,

# Save the FuncAnimation() to an object to ensure persistence
rick_ani = FuncAnimation(fig, animate, init_func=init, interval=100, blit=True)

plt.show()

In [ ]:
rick_ani.save('data/ricker_frequencies.gif', writer='imagemagick')

### Scrolling through a seismic cube

We use a different `matplotlib` function to scroll through images. As always, we refer to the [docs here](https://matplotlib.org/2.1.2/gallery/animation/dynamic_image2.html).

Here we create a `plt.figure()` object, then load our data, and append the images to a list that will be passed to the `ArtistAnimation()` function.

In [ ]:
# initialize a matplotlib figure
fig = plt.figure(figsize=(9,6))

# load the data
vol = np.load('data/Penobscot_0-1000ms.npy')
ma = np.percentile(vol, 99.0)
inline_range = np.arange(0, 601, 1)

"""
From the documentation:
ims is a list of lists, each row is a list of artists to draw in the
current frame; here we are just animating one artist, the image, in
each frame
"""
ims = []

# loop through all images and append to a list
for i in inline_range[::2]:
    plt.title(f'Penobscot')
    im = plt.imshow(vol[inline_range[i],:,:].T, aspect=1.0, vmin=-ma, vmax=ma, cmap='Greys', animated=True)
    ims.append([im])

# save the ArtistAnimation to an object to ensure persistence
seis_anim = ArtistAnimation(fig, ims, interval=10, blit=True)

plt.show()


#### Saving the animation

Saving the animation to a *.gif will work, but the *.mp4 is a lot smoother for such a large file.

In [ ]:
seis_anim.save('data/penobscot_scroller.gif', writer='imagemagick')

In [ ]:
seis_anim.save('data/penobscot_scroller.mp4')

### An SEG tutorial that makes a similar animation

For further examples in geophysics, it is worth following [this SEG tutorial](https://github.com/seg/tutorials-2017/blob/master/1712_FWI_forward_modeling/notebooks/manuscript.ipynb) that was published in 2017.

_nota bene_ this notebook depends on `devito`, [see the devito project page](https://www.devitoproject.org/) and running `devito` on Windows machines is currently not possible, or at least a big challenge.

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2019</p>
</div>